<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colabs/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pronto transformers datasets evaluate \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1

In [3]:
import pronto
from transformers import AutoTokenizer

In [4]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")

In [8]:
# Get subsumptions from CSV file to a dictionary

def get_subsumptions_from_file(filename):
  subsumptions = {}
  with open(filename) as f:
    for line in f:
      source_id, target_id = line.strip().split(',')
      subsumptions[source_id] = target_id
  return subsumptions

subs_doid_to_ncit = get_subsumptions_from_file("doid_to_ncit.csv")
subs_ncit_to_doid = get_subsumptions_from_file("ncit_to_doid.csv")

# Convert relations to sentences

> Currently considering parents, children & siblings for conceptual reasons, but could also take 'n-hop' appraoch, e.g. 1-hop only with parents and children, or 2-hop to include grandparents, grandchildren and siblings.

> How do I incorporate the desired mapping for training? Should I incorporate both all this AND target info, or too much? Could be SELF + desired relatives instead, or SELF + PARENT + DESIRED PARENT, etc.

In [5]:
# Create sentence from the given entity, containing its direct parents, children & siblings

def get_term_plus_relatives(entity_id, onto):
  markers = ["[SELF]", "[/SELF]", "[SUP]", "[/SUP]", "[SUB]", "[/SUB]", "[SIBL]", "[/SIBL]"]
  term_in, term_out, sup_in, sup_out, sub_in, sub_out, sibl_in, sibl_out = markers

  term = onto.get_term(entity_id)
  parents = term.superclasses(distance=1, with_self=False)
  children = term.subclasses(distance=1, with_self=False)
  siblings = set()
  for parent in parents:
    siblings.update(set(parent.subclasses(distance=1, with_self=False)))
  siblings.remove(term)

  term_plus_relatives = [term_in, term.name, term_out]
  for parent in parents:
    term_plus_relatives.extend([sup_in, parent.name, sup_out])
  for child in children:
    term_plus_relatives.extend([sub_in, child.name, sub_out])
  for sibling in siblings:
    term_plus_relatives.extend([sibl_in, sibling.name, sibl_out])

  return " ".join(term_plus_relatives)

In [6]:
print(get_term_plus_relatives("DOID:0050159", doid))

[SELF] lymphoid interstitial pneumonia [/SELF] [SUP] idiopathic interstitial pneumonia [/SUP] [SIBL] cryptogenic organizing pneumonia [/SIBL] [SIBL] desquamative interstitial pneumonia [/SIBL] [SIBL] nonspecific interstitial pneumonia [/SIBL] [SIBL] acute interstitial pneumonia [/SIBL]
